In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import random


# Depending on library versions on your system, one of the following imports 
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

In [5]:
path_to_datasets = r'dataset/'
target_img_size = (64, 64) # fix image size because classification algorithms THAT WE WILL USE HERE expect that

# We are going to fix the random seed to make our experiments reproducible 
# since some algorithms use pseudorandom generators
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

## Part I - Feature Extraction

In this part, we are going to implement two functions. Each one will extract a different set of features from the image. The two sets are:

1. Histogram of Gradients (HoG) features
2. Raw pixels (basically, not doing any feature extraction and just supplying the input image to the classifier)

In [6]:
def extract_hog_features(img):
    """
    TODO
    You won't implement anything in this function. You just need to understand it 
    and understand its parameters (i.e win_size, cell_size, ... etc)
    """
    img = cv2.resize(img, target_img_size)
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [7]:
def extract_raw_pixels(img):
    """
    TODO
    The classification algorithms we are going to use expect the input to be a vector not a matrix. 
    This is because they are general purpose and don't work only on images.
    CNNs, on the other hand, expect matrices since they operate on images and exploit the 
    arrangement of pixels in the 2-D space.
    
    So, what we only need to do in this function is to resize and flatten the image.
    """
    img = cv2.resize(img, target_img_size)

    img = img.flatten()

    return img

In [8]:
def extract_features(img, feature_set='hog'):
    """
    TODO
    Given either 'hsv_hist', 'hog', 'raw', call the respective function and return its output
    """
    if feature_set == 'hog':
        return extract_hog_features(img)

    if feature_set == 'raw':
        return extract_raw_pixels(img)

    return -1

The following function will extract the features and the label of each image in our dataset and save it in RAM. We normally don't save datasets in RAM, but this dataset is small.

In [9]:
def load_dataset(feature_set='hog'):
    features = []
    labels = []
    types = os.listdir(path_to_datasets)
    for t in types:
        path_to_dataset = f'{path_to_datasets}/{t}'
        img_filenames = os.listdir(path_to_dataset)
        for i, fn in enumerate(img_filenames):
            if fn.split('.')[-1] != 'jpg' and  fn.split('.')[-1] != 'png':
                continue

            labels.append(t)

            path = os.path.join(path_to_dataset, fn)
            img = cv2.imread(path)
            features.append(extract_features(img, feature_set))
            # if(i == 100): 
            #     break
            # show an update every 1,000 images
            if i > 0 and i % 1000 == 0:
                print("[INFO] processed {}/{}".format(i, len(img_filenames)))
        
    return features, labels        

## Part II - Classification

In this part, we will test the classification performance of SVM, KNN, & NNs given our features.

In [10]:
# TODO understand the hyperparameters of each classifier
classifiers = {
    'SVM': svm.LinearSVC(random_state=random_seed),
    'KNN': KNeighborsClassifier(n_neighbors=7),
    'NN': MLPClassifier(solver='sgd', random_state=random_seed, hidden_layer_sizes=(500,), max_iter=20, verbose=1)
}

In [11]:
# This function will test all our classifiers on a specific feature set
def run_experiment(feature_set):
    
    # Load dataset with extracted features
    print('Loading dataset. This will take time ...')
    features, labels = load_dataset(feature_set)
    print(len(labels))
    print('Finished loading dataset.')
    # Since we don't want to know the performance of our classifier on images it has seen before
    # we are going to withhold some images that we will test the classifier on after training 
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    
    for model_name, model in classifiers.items():
        print('############## Training', model_name, "##############")
        # Train the model only on the training features
        model.fit(train_features, train_labels)
        
        # Test the model on images it hasn't seen before
        accuracy = model.score(test_features, test_labels)
        
        print(model_name, 'accuracy:', accuracy*100, '%')

Now, we see how each classifier and each feature set performs

In [12]:
run_experiment('hog')

Loading dataset. This will take time ...
642
Finished loading dataset.
############## Training SVM ##############
SVM accuracy: 100.0 %
############## Training KNN ##############
KNN accuracy: 94.57364341085271 %
############## Training NN ##############
Iteration 1, loss = 2.46147119
Iteration 2, loss = 1.92034379
Iteration 3, loss = 1.44936613
Iteration 4, loss = 1.03428730
Iteration 5, loss = 0.72181444
Iteration 6, loss = 0.50891431
Iteration 7, loss = 0.37008317
Iteration 8, loss = 0.28187686
Iteration 9, loss = 0.21837134
Iteration 10, loss = 0.17567735
Iteration 11, loss = 0.14540049
Iteration 12, loss = 0.12313771
Iteration 13, loss = 0.10774574
Iteration 14, loss = 0.09391268
Iteration 15, loss = 0.08399710
Iteration 16, loss = 0.07482252
Iteration 17, loss = 0.06788022
Iteration 18, loss = 0.06205430
Iteration 19, loss = 0.05725713
Iteration 20, loss = 0.05311725
NN accuracy: 96.89922480620154 %


In [13]:
run_experiment('raw')

Loading dataset. This will take time ...
642
Finished loading dataset.
############## Training SVM ##############
SVM accuracy: 100.0 %
############## Training KNN ##############
KNN accuracy: 96.89922480620154 %
############## Training NN ##############
Iteration 1, loss = 31.91787267
Iteration 2, loss = 30.04082853
Iteration 3, loss = 35.55087287
Iteration 4, loss = 37.45742229
Iteration 5, loss = 21.66684662
Iteration 6, loss = 31.78272027
Iteration 7, loss = 41.22507397
Iteration 8, loss = 49.32062551
Iteration 9, loss = 56.24122713
Iteration 10, loss = 61.16584654
Iteration 11, loss = 65.05090292
Iteration 12, loss = 67.95016587
Iteration 13, loss = 70.10960998
Iteration 14, loss = 71.70790389
Iteration 15, loss = 72.84293295
Iteration 16, loss = 73.69923057
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
NN accuracy: 20.930232558139537 %


The classifiers list now has models trained on the last feature set you ran an experiment on. You can play around with it checking the probability it gives to each label, given an image.

In [26]:
# Example
test_img_path = r'test.png'
img = cv2.imread(test_img_path)
features = extract_features(img, 'raw')  # be careful of the choice of feature set

In [27]:
nn = classifiers['NN']
nn_probabilities =  nn.predict_proba([features])
nn_prediction =  nn.predict([features])
print('NN predicition:', nn_prediction)
print('NN probabilities: ', nn_probabilities)


NN predicition: ['natural']
NN probabilities:  [[0.08173966 0.07107601 0.07555108 0.07885446 0.08382439 0.07333052
  0.08306647 0.07768309 0.0700194  0.08118711 0.07988207 0.0675837
  0.07620204]]


In [28]:
knn = classifiers['KNN']
knn_probabilities =  knn.predict_proba([features])
knn_prediction =  knn.predict([features])
print('KNN predicition:', knn_prediction)
print('KNN probabilities: ', knn_probabilities)

KNN predicition: ['sharp']
KNN probabilities:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [29]:
svm = classifiers['SVM']
svm_prediction =  knn.predict([features])
print('SVM predicition:', svm_prediction)

SVM predicition: ['sharp']
